<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/MPI_OPenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install mpich libomp-dev

In [2]:
%%writefile mpi_openmp_e1.c

#include <stdio.h>
#include <stdlib.h>
//we load omp/mpi
#include <omp.h>
#include <mpi.h>
// defines the MPI_THREADS_MODE
#define MPI_THREAD_STRING(level)  \
        ( level==MPI_THREAD_SERIALIZED ? "THREAD_SERIALIZED" : \
                ( level==MPI_THREAD_MULTIPLE ? "THREAD_MULTIPLE" : \
                        ( level==MPI_THREAD_FUNNELED ? "THREAD_FUNNELED" : \
                                ( level==MPI_THREAD_SINGLE ? "THREAD_SINGLE" : "THIS_IS_IMPOSSIBLE" ) ) ) )

int main(int argc, char ** argv)
{
    /* Estos son los soportes de hilos deseados y disponibles.
        Se puede utilizar un código híbrido en el que todas las llamadas MPI se realizan desde el hilo principal (FUNNELED).
        Si los hilos realizan llamadas MPI, MULTIPLE es el apropiado. */
    int requested=MPI_THREAD_FUNNELED, provided;

    /* Intentamos activar los hilos MPI usando el modo requerido: MPI_THREAD_FUNNELED*/
    MPI_Init_thread(&argc, &argv, requested, &provided);
    if (provided<requested)
    {
        printf("MPI_Init_thread provee %s cuando %s fue solicitado.  Terminando el programa. \n",
               MPI_THREAD_STRING(provided), MPI_THREAD_STRING(requested) );
        exit(1);
    }

    int world_size, world_rank;

    MPI_Comm_size(MPI_COMM_WORLD,&world_size);
    MPI_Comm_rank(MPI_COMM_WORLD,&world_rank);

    printf("Hola desde %d de total :%d  procesos\n", world_rank, world_size);

    //ocupamos openMP para crear una seccion paralela
    #pragma omp parallel
    {
        int omp_id  =omp_get_thread_num();
        int omp_num =omp_get_num_threads();
        printf("MPI rank # %2d OpenMP thread # %2d of %2d \n", world_rank, omp_id, omp_num);
        fflush(stdout);
    }
    
    MPI_Finalize();
    return 0;
}

Overwriting mpi_openmp_e1.c


In [5]:
! ! mpicc -o mpi_openmp_e1 mpi_openmp_e1.c  -fopenmp 

In [ ]:
%env OMP_NUM_THREADS=3
! mpirun --allow-run-as-root -np 4 ./mpi_openmp_e1

## Matrix multiplication with MPI and OpenMP

In [35]:
%%writefile mpi_openmp_matrix_mult.c

/*
 * Un programa simple para multiplicar matrices
 * (Matrix_A  X  Matrix_B) => Matrix_C
 */
 
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <omp.h>

#include <time.h>


#define ARRAY_SIZE 10

typedef int matrix_t[ARRAY_SIZE][ARRAY_SIZE];

matrix_t MA,MB,MC;

/*
Rutina para multiplicar una fila por una columna y colocar un elemento en
matriz resultante.
*/
void mult(int size,
	  int row,
	  int column,
    int rowl,
	  matrix_t MA, 
	  matrix_t MB,
	  int *MC)
{
  int position;
  //int row_l=0;
  MC[rowl]= 0;
  for(position = 0; position < size; position++) {
     MC[rowl] = MC[rowl] +
     ( MA[row][position]  *  MB[position][column] ) ;

  }
}

//colocamos valores random 1-10 en las matrices
void inicializamos_matriz(int size, 
                    matrix_t MX)
{
    int   row, column;
    srand(time(0));
    for(row = 0; row < size; row++) {
    for (column = 0; column < size; column++) {
      MX[row][column]=rand()%10;
    }
  }
}

void imprimir_matriz(int size, 
                    matrix_t MX)
{
    int   row, column;
    for(row = 0; row < size; row ++) {
    for (column = 0; column < size; column++) {
      printf("%5d ",MX[row][column]);
    }
    printf("\n");
  }
}

void imprimir_matriz2(int sizer,int sizec, 
                    int *MX)
{
    int   row, column,i=0;
    for(row = 0; row < sizer; row++) {
    for (column = 0; column < sizec; column++) {
      printf("%5d ",MX[i]);
      i++;
    }
    printf("\n");
  }
}


// inicializamos valores y calcula los resultados

int main(void)
{
  int      size, row, column;
  
  size = ARRAY_SIZE;
  
//puntero a la matriz resultante
int *final_matrix;
int num_worker, rank;
MPI_Init(NULL, NULL);
MPI_Comm_size(MPI_COMM_WORLD, &num_worker);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);

if(rank == 0){
  // inicializamos los valores de la MA
  inicializamos_matriz(size, MA);
  //inicializamos los valores de la MB 
  inicializamos_matriz(size, MB);
  //imprimimos
  printf("La matriz A es;\n");
  imprimir_matriz(size,MA);
  printf("La matriz B es;\n");
  imprimir_matriz(size,MB);
  //reservamos la memoria para la matriz final
    final_matrix = (int *) malloc(sizeof(int*) * size*size);
}


MPI_Bcast(MA, size*size , MPI_INT, 0, MPI_COMM_WORLD);
MPI_Bcast(MB, size*size , MPI_INT, 0, MPI_COMM_WORLD);

//chequeamos si proceso 1 recibio la información
if(rank == 1){
  printf("id:%d La matriz A es;\n",rank);
  imprimir_matriz(size,MA);
  printf("id:%d La matriz B es;\n",rank);
  imprimir_matriz(size,MB);  
}

// determinamos la fila de inicio y fin para la proceso trabajador  
int startrow = rank * ( size / num_worker);
int endrow = ((rank + 1) * ( size / num_worker)) -1; 
//calculamos las sub-matrices
int number_of_rows = size / num_worker;
int *result_matrix = (int *) malloc(sizeof(int*) * number_of_rows * size);
    //multiplicamos
    int rowl=0;
    #pragma omp parallel for collapse(2)
    for(row = startrow;  row <= endrow; row++) {
     for (column = 0; column < size; column++) {
      mult(size, row, column,rowl, MA, MB, result_matrix);
      rowl++;
      int omp_id  =omp_get_thread_num();
      int omp_num =omp_get_num_threads();
      #pragma omp critical
      printf("row: %d col=%d, OpenMP_hilo=%d MPI_rank_process=%d\n", row,column, omp_get_thread_num(),rank);
      fflush(stdout);
      }
    }
   
   

//recolectamos los resutlados de la matriz
    MPI_Gather(result_matrix, number_of_rows*size, MPI_INT,
           final_matrix, number_of_rows*size,  MPI_INT, 0, MPI_COMM_WORLD);
  
  //imprimimos la matriz luego de recolectar los resultados
  if(rank == 0){
  printf("La matriz resultante C es (MPI/OpenMP);\n");  
  imprimir_matriz2(size,size,final_matrix);
  }

 MPI_Finalize();

  return 0;
}



Overwriting mpi_openmp_matrix_mult.c


In [36]:
 ! mpicc -o mpi_openmp_matrix_mult mpi_openmp_matrix_mult.c  -fopenmp 

In [ ]:
%env OMP_NUM_THREADS=3
! mpirun --allow-run-as-root -np 5 ./mpi_openmp_matrix_mult